In [ ]:
# DDGR20，嵌入近似hint，l-noisy < <v,s> < l+noisy
cd framework

In [ ]:
load("../framework/instance_gen.sage")
import numpy as np
import random

In [ ]:
n = 512
m = n
q = 3329
D_s = build_centered_binomial_law(3)
D_e = D_s
A, b, s, dbdd = initialize_from_LWE_instance(DBDD_predict, n, q, m, D_e, D_s)
print(s)
beta, delta = dbdd.estimate_attack()
_ = dbdd.integrate_q_vectors(q, report_every=20)

In [ ]:
def generate_se_eta_perfect_hint(m, n, q, k):
  V = []
  L = []

  for i in range(k):
    D_e = {-3: 1/64, -2: 6/64, -1: 15/64, 0: 20/64, 1: 15/64, 2: 6/64, 3: 1/64}
    values, probabilities = zip(*D_e.items())
    v = np.array(np.random.choice(values, size=m+n, p=probabilities))
    l = dbdd.leak(v)
    V.append(v)
    L.append(l)
  print("L",L)
  return V,L

In [ ]:
nph_Kyber128 = [0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 100, 150, 200, 250, 300, 350, 400, 450, 500, 550]
dis_Kyber128_our = [20.02, 20.01, 19.98, 19.9, 19.66, 19.59, 19.31, 19.14, 18.97, 18.79, 18.65, 16.51, 14.28, 12.37, 10.95, 9.4, 7.75, 4.07, 1.16, 0, 0]
nph_Kyber256 = [0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 120, 130, 140, 150, 160, 170, 180, 190, 200, 300, 400, 500, 600, 700, 800, 900, 1000, 1100]
dis_Kyber256_our = [27.35, 27.33, 27.3, 27.16, 27.02, 26.73, 26.57, 26.32, 25.95, 25.87, 25.5, 25.15, 25.03, 24.75, 24.43, 24.34, 24.02, 23.58, 23.41, 23.01, 22.84, 20.1, 17.2, 15.37, 13.71, 12.23, 9.43, 3.69, 0.11, 0]
nph_Kyber512 = [0, 20, 40, 60, 80, 100, 120, 140, 160, 180, 200, 220, 240, 260, 280, 300, 320, 340, 360, 380, 400, 420, 440, 460, 480, 500, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000, 2200]
dis_Kyber512_our = [38.48, 38.48, 38.33, 38.17, 37.8, 37.5, 37.19, 36.87, 36.55, 36.08, 35.71, 35.26, 35, 34.34, 34.05, 33.41, 32.81, 32.29, 31.96, 31.83, 31.67, 31.27, 30.83, 30.57, 30.03, 29.63, 27.35, 23.48, 20.67, 18.3, 15.79, 11.23, 2.93, 0, 0]

num_hint = 500
V, L = generate_se_eta_perfect_hint(m, n, q, num_hint)
BETA_emb = []
BETA_com = []
index = 0
for j in range(num_hint):
    if j == nph_Kyber512[index]:
        index += 1
        _ = dbdd.integrate_q_vectors(q, report_every=20)
        beta_emb, delta = dbdd.estimate_attack()
        print("beta_emb: ", beta_emb)
        BETA_emb.append(beta_emb)

        st = dis_Kyber512_our[index]/dis_Kyber512_our[0]
        beta_com, delta = dbdd.estimate_attack_SMY(st)
        print("beta_com: ", beta_com)
        BETA_com.append(beta_com)
    print("the ",j+1,"-th secret error perfect hint")
    _ = dbdd.integrate_perfect_hint(vec(V[j]), L[j])
print("BETA_emb", BETA_emb)
print("BETA_com", BETA_com)